# Bioclimatic data extraction
For each long-lat instance present in both the species' train and test set, the value of the following variables of interest are extracted:

- BIO1: Annual Mean Temperature
- BIO5: Max Temperature of Warmest Month
- BIO6: Min Temperature of Coldest Month
- BIO12: Annual Precipitation
- BIO15: Precipitation Seasonality (Coefficient of Variation)

### Preliminary code

In [1]:
# Importing useful libraries
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import xy
from rasterio.transform import rowcol
from pathlib import Path

### Checking data extraction system
**Process:**
1. Checking the coordinate system used by tiff, allowing to query a pixel and get the corresponding features for a location
2. Checking the coordinate system used by the training and test sets, to ensure that the mapping is coherent

In [2]:
# 1) Checking what coordinate system the tiff uses

input_files = ['wc2.1_10m_bio_1.tif', 'wc2.1_10m_bio_5.tif', 'wc2.1_10m_bio_12.tif', 'wc2.1_10m_bio_15.tif']

# Open the TIFF file
for file in input_files:
    with rasterio.open(file) as dataset:
    # Check the coordinate reference system
        crs = dataset.crs
        print(f"Coordinate Reference System: {crs}")

    # Check the transformation matrix (maps pixel to coordinates)
        transform = dataset.transform
        print(f"Transformation: {transform}")
# Example pixel coordinates (row, col)
row, col = 540, 1080

print("An example:")
# Get geographic coordinates (longitude, latitude)
with rasterio.open('wc2.1_10m_bio_6.tif') as dataset:
    lon, lat = xy(dataset.transform, row, col)
    print(f"Coordinates for pixel ({row}, {col}): Lon: {lon}, Lat: {lat}")
    pixel_value = dataset.read(1)[row, col]
    print(f"Pixel value at ({row}, {col}): {pixel_value}")

Coordinate Reference System: EPSG:4326
Transformation: | 0.17, 0.00,-180.00|
| 0.00,-0.17, 90.00|
| 0.00, 0.00, 1.00|
Coordinate Reference System: EPSG:4326
Transformation: | 0.17, 0.00,-180.00|
| 0.00,-0.17, 90.00|
| 0.00, 0.00, 1.00|
Coordinate Reference System: EPSG:4326
Transformation: | 0.17, 0.00,-180.00|
| 0.00,-0.17, 90.00|
| 0.00, 0.00, 1.00|
Coordinate Reference System: EPSG:4326
Transformation: | 0.17, 0.00,-180.00|
| 0.00,-0.17, 90.00|
| 0.00, 0.00, 1.00|
An example:
Coordinates for pixel (540, 1080): Lon: 0.08333333333331439, Lat: -0.0833333333333286
Pixel value at (540, 1080): -3.3999999521443642e+38


*They are all in the same reference system with the same transformation matrix (as expected)*

Wikipedia :EPSG:4326 - WGS 84, latitude/longitude coordinate system based on the Earth's center of mass, used by the Global Positioning System among others.

In [3]:
# 2) Checking what coordinate system the training and test sets use

# Loading training data    
data = np.load(Path('../species/species_train.npz'))
train_locs = data['train_locs']  # 2D array, rows are number of datapoints and columns are "latitude" and "longitude"
train_ids = data['train_ids'].astype(str)
    
# Checking that they also follow EPSG:4326
# Check the latitude and longitude ranges
latitudes = train_locs[:, 0]  # first column (latitude)
longitudes = train_locs[:, 1]  # second column (longitude)

# Check if latitudes and longitudes are within valid ranges
print((-90 <= latitudes).all() and (latitudes <= 90).all())
print((-180 <= longitudes).all() and (longitudes <= 180).all())


# Loading extra-training data    
extra_data = np.load(Path('../species/species_train_extra.npz'))
extra_train_locs = extra_data['train_locs']  # 2D array, rows are number of datapoints and columns are "latitude" and "longitude"
extra_train_ids = extra_data['train_ids'].astype(str)
    
# Checking that they also follow EPSG:4326
# Check the latitude and longitude ranges
extra_latitudes = extra_train_locs[:, 0]  # first column (latitude)
extra_longitudes = extra_train_locs[:, 1]  # second column (longitude)

# Check if latitudes and longitudes are within valid ranges
print((-90 <= extra_latitudes).all() and (extra_latitudes <= 90).all())
print((-180 <= extra_longitudes).all() and (extra_longitudes <= 180).all())


# Loading test data
data_test = np.load(Path('../species/species_test.npz'), allow_pickle=True)
test_locs = data_test['test_locs']

# Checking that they also follow EPSG:4326
# Check the latitude and longitude ranges
latitudes = test_locs[:, 0]  # first column (latitude)
longitudes = test_locs[:, 1]  # second column (longitude)

# Check if latitudes and longitudes are within valid ranges
print((-90 <= latitudes).all() and (latitudes <= 90).all())
print((-180 <= longitudes).all() and (longitudes <= 180).all())

True
True
True
True
True
True


### Extracting data

In [4]:
# Input files corresponding to the BIO variables
input_files = ['wc2.1_10m_bio_1.tif',  # BIO1: Annual Mean Temperature
               'wc2.1_10m_bio_5.tif',  # BIO5: Max Temperature of Warmest Month
               'wc2.1_10m_bio_6.tif',  # BIO6 = Min Temperature of Coldest Month
               'wc2.1_10m_bio_12.tif', # BIO12: Annual Precipitation
               'wc2.1_10m_bio_15.tif'] # BIO15: Precipitation Seasonality (Coefficient of Variation)

In [ ]:
# Extracting train-set's bioclimatic data

# Empty list to store the values from each TIFF file
mapped_values = np.zeros((train_locs.shape[0], len(input_files)))

for idx, tiff_file in enumerate(input_files):
    with rasterio.open(tiff_file) as dataset:
        transform = dataset.transform
        
        # Convert lat/lon coordinates to (row, col) pixel coordinates for all locations
        lon_lats = train_locs[:, ::-1]  # reverse to get (lon, lat) for rasterio -> different requirements
        rows_cols = np.array([rowcol(transform, lon, lat) for lon, lat in lon_lats])
        
        # Extract the pixel values from the raster for each (row, col)
        for i, (row, col) in enumerate(rows_cols):
            row, col = int(row), int(col) # potential issues
            
            # Read the value from the TIFF file at the corresponding pixel
            mapped_values[i, idx] = dataset.read(1)[row, col]  # layer/band 1
        print("done layer")

column_names = ['lat', 'long', 'BIO1', 'BIO5', 'BI06', 'BIO12', 'BIO15']
train_locs_extended = pd.DataFrame(np.hstack((train_locs, mapped_values)), columns=column_names)

train_locs_extended.insert(0, 'id', train_ids) # insert also the ids column

# Display and save the dataset
display(train_locs_extended.head())
train_locs_extended.to_csv('bioclimatic_train.csv', index=False)

done layer
done layer
done layer
done layer
done layer


,id,lat,long,BIO1,BIO5,BI06,BIO12,BIO15
0,31529,-18.286728,143.481247,25.830833,36.812252,12.26175,778.0,118.931313
1,31529,-13.099798,130.783646,26.970772,35.256001,15.94425,1637.0,106.943298
2,31529,-13.965274,131.695145,27.042313,37.103500,15.02150,1190.0,112.085449
3,31529,-12.853950,132.800507,27.847281,36.489750,17.33075,1438.0,111.659103
4,31529,-12.196790,134.279327,27.310499,35.029251,18.08950,1250.0,112.806938


In [5]:
# Extracting extra-train-set's bioclimatic data

# Empty list to store the values from each TIFF file
mapped_values = np.zeros((extra_train_locs.shape[0], len(input_files)))

for idx, tiff_file in enumerate(input_files):
    with rasterio.open(tiff_file) as dataset:
        transform = dataset.transform
        
        # Convert lat/lon coordinates to (row, col) pixel coordinates for all locations
        lon_lats = extra_train_locs[:, ::-1]  # reverse to get (lon, lat) for rasterio -> different requirements
        rows_cols = np.array([rowcol(transform, lon, lat) for lon, lat in lon_lats])
        
        # Extract the pixel values from the raster for each (row, col)
        for i, (row, col) in enumerate(rows_cols):
            row, col = int(row), int(col) # potential issues
            
            # Read the value from the TIFF file at the corresponding pixel
            mapped_values[i, idx] = dataset.read(1)[row, col]  # layer/band 1
        print("done layer")

column_names = ['lat', 'long', 'BIO1', 'BIO5', 'BI06', 'BIO12', 'BIO15']
train_locs_extended = pd.DataFrame(np.hstack((extra_train_locs, mapped_values)), columns=column_names)

train_locs_extended.insert(0, 'id', extra_train_ids) # insert also the ids column

# Display and save the dataset
display(train_locs_extended.head())
train_locs_extended.to_csv('bioclimatic_train_extra.csv', index=False)

done layer
done layer
done layer
done layer
done layer


,id,lat,long,BIO1,BIO5,BI06,BIO12,BIO15
0,14,-22.128679,-46.795666,19.809572,27.388250,8.83025,1519.0,71.723351
1,14,-22.833548,-47.105415,20.322948,27.775499,10.92950,1295.0,67.242844
2,14,-20.454288,-54.581146,23.133968,30.171249,13.95675,1450.0,54.357277
3,14,-22.676571,-45.841366,14.146209,23.919750,2.80600,1779.0,65.925346
4,14,-22.310076,-42.497063,17.898302,26.118500,8.12775,1342.0,68.509933


In [13]:
# Extracting test-set's bioclimatic data

# Input files corresponding to the BIO variables (add any input files that you want)
input_files = ['wc2.1_10m_bio_1.tif',  # BIO1: Annual Mean Temperature
               'wc2.1_10m_bio_5.tif',  # BIO5: Max Temperature of Warmest Month
               'wc2.1_10m_bio_6.tif',  # BIO6 = Min Temperature of Coldest Month
               'wc2.1_10m_bio_12.tif', # BIO12: Annual Precipitation
               'wc2.1_10m_bio_15.tif'] # BIO15: Precipitation Seasonality (Coefficient of Variation)

# Empty list to store the values from each TIFF file
mapped_values = np.zeros((test_locs.shape[0], len(input_files)))

for idx, tiff_file in enumerate(input_files):
    with rasterio.open(tiff_file) as dataset:
        transform = dataset.transform
        
        # Convert lat/lon coordinates to (row, col) pixel coordinates for all locations
        lon_lats = test_locs[:, ::-1]  # reverse to get (lon, lat) for rasterio -> different requirements
        rows_cols = np.array([rowcol(transform, lon, lat) for lon, lat in lon_lats])
        
        # Extract the pixel values from the raster for each (row, col)
        for i, (row, col) in enumerate(rows_cols):
            row, col = int(row), int(col) # potential issues
            
            # Read the value from the TIFF file at the corresponding pixel
            mapped_values[i, idx] = dataset.read(1)[row, col]  # layer/band 1
        print("done layer")

column_names = ['lat', 'long', 'BIO1', 'BIO5', 'BI06', 'BIO12', 'BIO15']
test_locs_extended = pd.DataFrame(np.hstack((test_locs, mapped_values)), columns=column_names)

# Display and save the dataset
display(test_locs_extended.head())
test_locs_extended.to_csv('bioclimatic_test.csv', index=False)

done layer
done layer
done layer
done layer
done layer


,lat,long,BIO1,BIO5,BI06,BIO12,BIO15
0,9.630478,-173.535599,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
1,3.839375,-162.544464,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
2,4.289169,-167.944778,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
3,3.879849,-169.720459,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
4,-6.237210,-169.554123,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38


In [ ]:
# Adding ids to each test location
bioclimatic_test = pd.read_csv('bioclimatic_test.csv', dtype={'id': str, 'lat':np.float32, 'long':np.float32})
display(bioclimatic_test)
species_test = np.load(Path('../species/species_test.npz'), allow_pickle=True)
taxon_ids = species_test['taxon_ids'].astype(str)    # 1D array of each location's relevant specie IDs
test_locations = species_test['test_locs'].astype(np.float64)            # 2D array with each datapoint's latitude and longitude
test_latitudes = test_locations[:,0]
test_longitudes = test_locations[:,1]
test_pos_inds = species_test['test_pos_inds']                      # list of lists, each list corresponds to the indices in test_locs where a given species is present
                                                    # > it can be assumed that they are not present in the other locations 
test_pos_inds = dict(zip(taxon_ids, test_pos_inds))
rows = []
for taxon_id,indices in test_pos_inds.items():
    for idx in indices:
        lat,long = test_locations[idx]
        lat,long = np.float32(lat),np.float32(long)
        if len(bioclimatic_test[(bioclimatic_test['lat']==lat)&(bioclimatic_test['long']==long)].to_dict(orient='records'))!=1: print(len(bioclimatic_test[(bioclimatic_test['lat']==lat)&(bioclimatic_test['long']==long)].to_dict(orient='records')))
        row = {'id': taxon_id}
        row.update(bioclimatic_test[(bioclimatic_test['lat']==lat)&(bioclimatic_test['long']==long)].to_dict(orient='records')[0])
        row.update({'lat':test_latitudes[idx],'long':test_longitudes[idx]})
        rows.append(row)
bioclimatic_test = pd.DataFrame(rows)
display(bioclimatic_test)
bioclimatic_test.to_csv('bioclimatic_test.csv', index=False)

,lat,long,BIO1,BIO5,BI06,BIO12,BIO15
0,9.630478,-173.535599,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
1,3.839375,-162.544464,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
2,4.289169,-167.944778,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
3,3.879849,-169.720459,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
4,-6.237210,-169.554123,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
...,...,...,...,...,...,...,...
288117,-23.468565,110.252289,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
288118,-18.319242,107.135307,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
288119,-21.390581,111.551872,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
288120,-21.996183,109.184601,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38


,id,lat,long,BIO1,BIO5,BI06,BIO12,BIO15
0,31529,-19.884237,126.052979,27.631907,40.141750,12.100750,409.0,108.091408
1,31529,-20.219316,124.723953,27.002522,39.909500,11.117000,383.0,110.598808
2,31529,-20.053690,125.386505,27.538322,40.171249,11.937750,381.0,110.677689
3,31529,-19.973000,126.462440,27.495510,40.019001,11.996500,419.0,105.716896
4,31529,-19.962839,124.980362,27.671824,40.339500,11.784500,379.0,116.957329
...,...,...,...,...,...,...,...,...
1706641,145031,31.998211,72.541458,23.857594,41.209000,3.896750,371.0,104.039230
1706642,145031,26.927755,69.225052,27.683115,44.576248,6.472500,122.0,152.254761
1706643,145031,23.349318,70.605515,26.850750,39.846001,10.351500,481.0,159.469482
1706644,145031,23.706282,68.259659,26.218336,35.095634,12.511905,261.0,164.298828


In [ ]:
# Cleaning bioclimatic data (-3.400000e+38 is placeholder for not found value, so filtering out basing on one varaible)
bioclimatic_train = pd.read_csv('bioclimatic_train.csv', dtype={'id': str})
bioclimatic_train_extra = pd.read_csv('bioclimatic_train_extra.csv', dtype={'id': str})
bioclimatic_test = pd.read_csv('bioclimatic_test.csv', dtype={'id': str})
bioclimatic_train = bioclimatic_train[bioclimatic_train['BIO1']>=-1000]
bioclimatic_train_extra = bioclimatic_train_extra[bioclimatic_train_extra['BIO1']>=-1000]
bioclimatic_test = bioclimatic_test[bioclimatic_test['BIO1']>=-1000]
display(bioclimatic_train)
display(bioclimatic_train_extra)
display(bioclimatic_test)
bioclimatic_train.to_csv('cleaned_bioclimatic_train.csv', index=False)
bioclimatic_train_extra.to_csv('cleaned_bioclimatic_train_extra.csv', index=False)
bioclimatic_test.to_csv('cleaned_bioclimatic_test.csv', index=False)

,id,lat,long,BIO1,BIO5,BI06,BIO12,BIO15
0,31529,-18.286728,143.481247,25.830833,36.812252,12.26175,778.0,118.931313
1,31529,-13.099798,130.783646,26.970772,35.256001,15.94425,1637.0,106.943298
2,31529,-13.965274,131.695145,27.042313,37.103500,15.02150,1190.0,112.085449
3,31529,-12.853950,132.800507,27.847281,36.489750,17.33075,1438.0,111.659103
4,31529,-12.196790,134.279327,27.310499,35.029251,18.08950,1250.0,112.806938
...,...,...,...,...,...,...,...,...
272032,145031,33.716885,73.203621,20.198093,36.904251,3.22975,1087.0,85.595444
272033,145031,24.600239,72.730560,23.997032,37.051250,9.97050,1013.0,166.995056
272034,145031,18.849600,80.654129,26.365208,39.693748,13.54425,1547.0,135.629791
272035,145031,21.073837,75.945656,27.160301,41.581749,13.43450,735.0,124.973358


,id,lat,long,BIO1,BIO5,BI06,BIO12,BIO15
0,14,-22.128679,-46.795666,19.809572,27.388250,8.830250,1519.0,71.723351
1,14,-22.833548,-47.105415,20.322948,27.775499,10.929500,1295.0,67.242844
2,14,-20.454288,-54.581146,23.133968,30.171249,13.956750,1450.0,54.357277
3,14,-22.676571,-45.841366,14.146209,23.919750,2.806000,1779.0,65.925346
4,14,-22.310076,-42.497063,17.898302,26.118500,8.127750,1342.0,68.509933
...,...,...,...,...,...,...,...,...
1067587,1369303,3.339513,101.247047,27.008966,32.121464,22.315968,2360.0,28.365269
1067588,1369303,3.176027,101.830063,25.754875,31.064751,20.930750,2496.0,27.776682
1067589,1369303,3.339002,101.244804,27.008966,32.121464,22.315968,2360.0,28.365269
1067590,1369303,3.342500,101.246635,27.008966,32.121464,22.315968,2360.0,28.365269


,id,lat,long,BIO1,BIO5,BI06,BIO12,BIO15
0,31529,-19.884237,126.052979,27.631907,40.141750,12.100750,409.0,108.091408
1,31529,-20.219316,124.723953,27.002522,39.909500,11.117000,383.0,110.598808
2,31529,-20.053690,125.386505,27.538322,40.171249,11.937750,381.0,110.677689
3,31529,-19.973000,126.462440,27.495510,40.019001,11.996500,419.0,105.716896
4,31529,-19.962839,124.980362,27.671824,40.339500,11.784500,379.0,116.957329
...,...,...,...,...,...,...,...,...
1706641,145031,31.998211,72.541458,23.857594,41.209000,3.896750,371.0,104.039230
1706642,145031,26.927755,69.225052,27.683115,44.576248,6.472500,122.0,152.254761
1706643,145031,23.349318,70.605515,26.850750,39.846001,10.351500,481.0,159.469482
1706644,145031,23.706282,68.259659,26.218336,35.095634,12.511905,261.0,164.298828
